In [ ]:
import logging
import math
import random
from collections import defaultdict
from functools import partial

import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import open3d as o3d
import torch
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    CamerasBase,
    FoVPerspectiveCameras,
    MeshRasterizer,
    RasterizationSettings,
    look_at_view_transform,
)
from pytorch3d.structures import Meshes
from neural_poisson.data.prepare import load_mesh, extract_surface_data

model_idx = 9
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCore/04256520"
model_id = sorted(list(Path(shapenet_dir).iterdir()))[model_idx].stem
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

R, T = look_at_view_transform(1.0, 10, 180)
camera = FoVPerspectiveCameras(device="cuda", R=R, T=T, fov=60.0)

mesh = load_mesh(shapenet_path, "cuda")

data = extract_surface_data(
    camera=camera,
    mesh=mesh,
    image_size=512,
    fill_depth="max2",
)

normals = (data["normal_map"] + 1) / 2
plt.imshow(normals.detach().cpu().numpy())

In [ ]:
from neural_poisson.data.grid import coord_grid, coord_grid_along_axis

voxel_size = 256
domain = (-1.0, 1.0)
grid = coord_grid_along_axis(voxel_size=voxel_size, domain=domain)
grid.shape

In [ ]:
import numpy as np
from neural_poisson.data import binvox_rw

with open(shapenet_dir / model_id / "models/model_normalized.surface.binvox", 'rb') as f:
    model = binvox_rw.read_as_3d_array(f)

In [ ]:
# Convert to Open3D VoxelGrid
voxel_size = 1.0
origin = np.array([0, 0, 0])  # Set origin if needed

voxel_grid = o3d.geometry.VoxelGrid.create_from_dense(
    model.data.astype(np.float32),  # Convert boolean array to float
    voxel_size=voxel_size,
    origin=origin
)

# Visualize
o3d.visualization.draw_geometries([voxel_grid])

In [ ]:
import open3d as o3d
# Extract occupied voxel indices
voxel_indices = np.argwhere(m1.data)  # Get indices of occupied voxels

# Convert to Open3D voxel grid
voxel_size = 1.0  # Adjust voxel size if needed
voxel_grid = o3d.geometry.VoxelGrid()

# for voxel in voxel_indices:
#     voxel_grid.add_voxel(o3d.geometry.Voxel(o3d.utility.Vector3iVector([voxel]), voxel_size))

voxel_grid.add_voxel()

In [ ]:
import open3d as o3d

# Create Open3D PointCloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data["points"].detach().cpu().numpy())

# Convert PointCloud to VoxelGrid
voxel_size = 0.01  # Adjust as needed
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)
o3d.visualization.draw_plotly([voxel_grid])

In [ ]:
from neural_poisson.mesh.utils import pytorch3d_to_open3d
import open3d as o3d

_mesh = pytorch3d_to_open3d(mesh)
o3d.visualization.draw_plotly([_mesh])

In [ ]:
list(Path(shapenet_dir).iterdir())[0]

In [ ]:
normals.min()

In [ ]:
 list(Path(shapenet_dir).iterdir())

In [ ]:
list(shapenet_dir.iterdir())